# Student Performance: Predicting Student's Math Score

The goal of this notebook is to apply what I have previously learned from courses with regards to machine learning. For this, we will be evaluating the dataset of student performance and create a conclusive statement as to how features presented in the dataset could affect the math scores of our students.

Below we have written the necessary Python code for setting up our notebook, which includes the importation of the necessary libraries.

In [23]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plot
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input/students-performance-in-exams'):
    for filename in filenames:
        file_path = (os.path.join(dirname, filename))
        dataframe = pd.read_csv(file_path)
        dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/kaggle/input/students-performance-in-exams'):
    for filename in filenames:
        print(os.path.join(dirname, filename))